In [1]:
#import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pylab as pl
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import MiniBatchKMeans
from preprocess import *
from collections import Counter
import cv2
import math
from skimage.transform import pyramid_reduce
from sklearn.metrics import accuracy_score
%load_ext autoreload
%autoreload 2

In [2]:
with open('fonts-dataset/font_names.txt') as file:
    lines = file.readlines()
    styles = [line.rstrip()[4:] for line in lines] # ignoring number at first of each style (first 4 chars)
print(styles)

['ScheherazadeNew', 'marhey', 'lemonada', 'ibm']


In [3]:
x=[]
y=[]
for i in range(0, len(styles)):
    for filename in sorted(glob.glob('fonts-dataset/'+str(i)+'/*.jpeg')):
        img = cv2.imread(filename)
     #   resized_img = cv2.resize(img, (200, 200))
        x.append(img)
        y.append(i)



In [4]:

# x = np.asarray(x)
# y = np.asarray(y)
#print ("The size of the training set is: ", x.shape)
#print ("The size of the labels of the training set is: ", y.shape)
print (y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
x_train_preprocessed = []
y_train_modified=[]
print(len(x_train))
print(len(y_train))
i=0
for img in x_train:
    img = threshold_image(img)
    img= assure_white_bg(img)
    img=get_words(img)
    x_train_preprocessed.extend(img)
    y_train_modified.extend([y_train[i]]*len(img))
    i=i+1

3200
3200


In [6]:
A_moms=[]
i=0
for im in x_train_preprocessed:
    moments = cv2.moments(im) 
    huMoments = cv2.HuMoments(moments)
    huMoments = -np.sign(huMoments) * np.log10(np.abs(huMoments+0.001))
    A_moms.append(huMoments.flatten())

In [10]:
print(len(y_train_modified))
print(len(A_moms))
from classifiers_manager import ClassifiersManager,Classifiers
classifiers_manager = ClassifiersManager()
classifiers_manager.train(Classifiers.svm,A_moms,y_train_modified)

220506
220506


In [ ]:
x_test_preprocessed = []
y_test_modified=[]
i=0
for img in x_test:
    img = threshold_image(img)
    img= assure_white_bg(img)
    img=get_words(img)
    x_test_preprocessed.append(img)
    y_test_modified.extend([y_test[i]]*len(img))
    i=i+1

In [ ]:
print(len(x_test_preprocessed))
A_Tmoms=[]
for im in x_test_preprocessed:
    temp=[]
    for i in im:
        moments = cv2.moments(i) 
        huMoments = cv2.HuMoments(moments)
        huMoments = -np.sign(huMoments) * np.log10(np.abs(huMoments+0.001))
        temp.append(huMoments.flatten())
    A_Tmoms.append(temp)

1


In [ ]:
print(len(A_Tmoms))
print(len(y_test_modified))
A_Tmoms= np.array(A_Tmoms.reshape(-1,7))
y_test_modified=np.array(y_test_modified)
print(A_Tmoms.shape)
print(y_test_modified.shape)
classifiers_manager.test_all_classifiers(A_Tmoms,y_test_modified,styles)
classifiers_manager.print_metrics()

135
135
(135, 7)
(135,)
SVM
                 precision    recall  f1-score   support

ScheherazadeNew       0.98      1.00      0.99       132
         marhey       0.00      0.00      0.00         1
       lemonada       0.00      0.00      0.00         1
            ibm       0.00      0.00      0.00         1

       accuracy                           0.98       135
      macro avg       0.24      0.25      0.25       135
   weighted avg       0.96      0.98      0.97       135

KNN
                 precision    recall  f1-score   support

ScheherazadeNew       0.98      1.00      0.99       132
         marhey       0.00      0.00      0.00         1
       lemonada       0.00      0.00      0.00         1
            ibm       0.00      0.00      0.00         1

       accuracy                           0.98       135
      macro avg       0.24      0.25      0.25       135
   weighted avg       0.96      0.98      0.97       135

Random_Forest
                 precision    recall

C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le